In [ ]:
import os
os.environ["FI_PROVIDER"] = "tcp"
os.environ["MPICH_OFI_STARTUP_CONNECT"] = "0"
 

from mpi4py import MPI
import numpy as np
import matplotlib.pyplot as plt
from petsc4py.PETSc import ScalarType
from dolfinx import mesh, fem, plot, io, la
from dolfinx.fem.petsc import LinearProblem
import ufl
import pyvista
import pyvistaqt


# Mesh
msh = mesh.create_rectangle(
    comm=MPI.COMM_WORLD,
    points=((0.0, 0.0), (1.0, 1.0)),
    n=(64, 64),
    cell_type=mesh.CellType.triangle,
)

dt = 0.1
t = fem.Constant(msh, ScalarType(0))
num_time_steps = 100


# Function space
V = fem.functionspace(msh, ("Lagrange", 1))
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)
 

u_old = fem.Function(V)
u_old.x.array[:] = 0.0


# Dicichlet boundary conditions
def marker(x):
    return (
        np.isclose(x[0], 0.0)
        | np.isclose(x[1], 0.0)
        | np.isclose(x[0], 1.0)
        | np.isclose(x[1], 1.0)
    )


facets = mesh.locate_entities_boundary(msh, dim=(msh.topology.dim - 1), marker=marker)
dofs = fem.locate_dofs_topological(V=V, entity_dim=1, entities=facets)
bc = fem.dirichletbc(value=ScalarType(0), dofs=dofs, V=V)


# RHS function
x = ufl.SpatialCoordinate(msh)
f = 2.0 * t * (x[0] * (1.0 - x[0]) + x[1] * (1.0 - x[1])) + x[0] * x[1] * (1.0 - x[0]) * (1.0 - x[1])

u_exact = x[0] * x[1] * (1.0 - x[0]) * (1.0 - x[1]) * t

# Variational form
a = ufl.inner(u, v) * ufl.dx + dt * ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx
L = dt * ufl.inner(f, v) * ufl.dx + ufl.inner(u_old, v) * ufl.dx

 

# Problem

problem = LinearProblem(
    a,
    L,
    bcs=[bc],
    petsc_options_prefix="test_poisson_",
    petsc_options={
        "ksp_type": "preonly",
        "pc_type": "lu",
        "ksp_error_if_not_converged": True,
    },
)



###############################
# Plotting with pyvista
###############################

uh = problem.solve()

print(type(uh))

cells, types, x = plot.vtk_mesh(V)
grid = pyvista.UnstructuredGrid(cells, types, x)
grid.point_data["uh"] = uh.x.array[:].real
grid.set_active_scalars("uh")
 

p = pyvistaqt.BackgroundPlotter(title="u", auto_update=True)
p.add_mesh(grid, clim =[0,0.5])
p.view_xy(True)
p.add_text(f"time: {t}", font_size=12, name="timelabel")



for i in range(num_time_steps):
    t.value += dt
    uh = problem.solve()
    u_old.x.array[:] = uh.x.array[:]
    l2_error = np.sqrt(
    fem.assemble_scalar(fem.form(ufl.inner(uh - u_exact, uh - u_exact) * ufl.dx))
    )
    print(f"Manual norm: {l2_error}")

    # Add scalar data to grid
    p.add_text(f"time: {t.value:.2e}", font_size=12, name="timelabel")
    grid.point_data["uh"] = uh.x.array.real
    p.app.processEvents()




with io.XDMFFile(msh.comm, "output/poisson.xdmf", "w") as file:
    file.write_mesh(msh)
    file.write_function(uh)

<class 'dolfinx.fem.function.Function'>


/Users/wilhelmfaerden/miniforge3/envs/fenicsx-env/lib/python3.11/site-packages/pyvista/plotting/plotter.py:1653: PyVistaDeprecationWarning: 
../../miniforge3/envs/fenicsx-env/lib/python3.11/site-packages/pyvista/plotting/plotter.py:1653: Argument 'negative' must be passed as a keyword argument to function 'Renderer.view_xy'.
From version 0.50, passing this as a positional argument will result in a TypeError.
  self.renderer.view_xy(*args, **kwarg)


Manual norm: 1.7140805510714884e-06
Manual norm: 3.805271688888694e-06
Manual norm: 6.028362640172539e-06
Manual norm: 8.297384101887187e-06
Manual norm: 1.0582515326579233e-05
Manual norm: 1.2873423985234117e-05
Manual norm: 1.516649880657088e-05
Manual norm: 1.7460450949545882e-05
Manual norm: 1.975480208091528e-05
Manual norm: 2.2049362567893916e-05
Manual norm: 2.4344049416519656e-05
Manual norm: 2.663882143981514e-05
Manual norm: 2.893365535278061e-05
Manual norm: 3.1228536453455036e-05
Manual norm: 3.352345468276907e-05
Manual norm: 3.5818402776665476e-05
Manual norm: 3.8113375296475584e-05
Manual norm: 4.040836806530991e-05
Manual norm: 4.2703377813306954e-05
Manual norm: 4.499840194079653e-05
Manual norm: 4.729343835375266e-05
Manual norm: 4.958848534621565e-05
Manual norm: 5.188354151415124e-05
Manual norm: 5.417860569153301e-05
Manual norm: 5.647367690182979e-05
Manual norm: 5.876875432106388e-05
Manual norm: 6.106383724914216e-05
Manual norm: 6.33589250874626e-05
Manual norm

: 